In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline
import time

In [ ]:
#to keep things in order, and to avoid to copy and paste everytime our functions if we want to use them in more than one folder,
#we can temporarily use this library. 
import sys

#in this way Python will search the implementations also in the path '../HelperFunctions'
sys.path.insert(0, '../HelperFunctions')
sys.path.insert(0, '../pre-processing/Clean_Data/')

In [ ]:
from proj1_helpers import *
from common_functions import *
from counters import *
from replace import *
from regressors import batch_iter

In [ ]:
yb, input_data, ids = load_csv_data("../data/train.csv", sub_sample=False)

In [ ]:
#this will surely be deleted, in this way we are sure that original_data is the original version of the data and we don't have
#to load them again
from copy import deepcopy
originalData = deepcopy(input_data)
originalY = deepcopy(yb)

# Functions from lab 5

In [ ]:
def sig(t):
    """apply sigmoid function on t."""
    if t > 0:
        return 1 / (1 + np.exp(-t))
    else:
        return np.exp(t) / (1 + np.exp(t))

sigmoid = np.vectorize(sig)

def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood."""
    pred = sigmoid(tx.dot(w))
    loss = y.T.dot(np.log(pred)) + (1 - y).T.dot(np.log(1 - pred))
    return np.squeeze(- loss)


def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    pred = sigmoid(tx.dot(w))
    grad = tx.T.dot(pred - y)
    return grad

def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descen using logistic regression.
    Return the loss and the updated w.
    """
    #batch = next(batch_iter(y, tx, 32))
    #minibatch_y, minibatch_tx = batch[0], batch[1]
    grad = calculate_gradient(y, tx, w)
    loss = calculate_loss(y, tx, w)
    w -= gamma * grad
    return loss, w

In [ ]:
#Creation of tx

input_data = deepcopy(originalData)

#Clean the dataset
numInvalidValues = countInvalid(input_data,-999)
idxCols = np.where(numInvalidValues>0)[0]
input_data = replaceWithZero(input_data,-999,idxCols)

#standardize
tx,_,_ = standardize(input_data)

#add ones
tx = np.c_[np.ones((yb.shape[0], 1)), tx]
y = yb

In [ ]:
#first tries with gradient descent

t, tx = sample_data(y, tx, 1, 500)
max_iter = 1000
w = np.zeros(tx.shape[1])
gamma = 0.004
loss0 = 0


for iter in range(max_iter):
    loss, w = learning_by_gradient_descent(y, tx, w, gamma)
    if iter%50==0:
        print(loss)
    if iter%20:
        gamma = gamma * 0.5
    loss0 = loss




Let's try to generate a submission

In [ ]:
WWW = w 

In [ ]:
xxx, test_data, ids = load_csv_data("../data/test.csv", sub_sample=False)

In [ ]:
#we need to preprocess the test_data as we preprocessed the train data

numInvalidValues = countInvalid(test_data,-999)
idxCols = np.where(numInvalidValues>0)[0]
test_data = replaceWithZero(test_data,-999,idxCols)
txTest,_,_ = standardize(test_data)
txTest = np.c_[np.ones((xxx.shape[0], 1)), txTest]

In [ ]:
y_pred = predict_labels(WWW, txTest)

In [ ]:
create_csv_submission(ids, y_pred, 'submission.csv')

# Same things with PCA

In [ ]:
sys.path.insert(0, '../pre-processing/PCA/')

In [ ]:
from pca_functions import PCAWithCovariance

input_data = deepcopy(originalData)

numInvalidValues = countInvalid(input_data,-999)
idxCols = np.where(numInvalidValues>0)[0]
input_data = replaceWithZero(input_data,-999,idxCols)
input_data,_,_ = standardize(input_data)

In [ ]:
#Generating the principal components

_,eV = PCAWithCovariance(input_data)

N = 6 #num p. components
components = np.empty(input_data.shape[0])
for i in range(N):
    components = np.c_[components, input_data.dot(eV[:,i])]
    
tx = np.c_[np.ones(input_data.shape[0]), components]

In [ ]:
Y, X = sample_data(y, tx, 1, 500)

In [ ]:
max_iter = 1000
w = np.zeros(X.shape[1])
gamma = 0.0004
loss0 = 0
for iter in range(max_iter):
    loss, w = learning_by_gradient_descent(Y, X, w, gamma)
    if iter%50==0:
        print(f'looooosssss= {loss}')
    loss0 = loss